# Calculating the Soil Wetness Index

In this example we will plot the Soil Moisture as done on the EFAS Website

First we open the 6 static files and the 1 data file for the ECMWF HRES Model.

We will only be working on Layer 1 and 2 in this example.

[Soil Wetness Index Calculation](https://confluence.ecmwf.int/display/COPSRV/Soil+wetness+index+calculation)

In [ ]:
import numpy as np
import xarray as xr

thmin1 = xr.open_dataset('../static/thmin1.nc')
thmin2 = xr.open_dataset('../static/thmin2.nc')
thmin3 = xr.open_dataset('../static/thmin3.nc')
thmax1 = xr.open_dataset('../static/thmax1.nc')
thmax2 = xr.open_dataset('../static/thmax2.nc')
thmax3 = xr.open_dataset('../static/thmax3.nc')
ds = xr.open_dataset('../data/eud.nc')


We have 3 layers of Volumetric Soil Moisture which we need to get the mean across all timesteps

In [ ]:
import warnings; warnings.simplefilter('ignore')
mean_vsw = ds.vsw.mean('step') # Mean Volumetric Soil Moisture for all time steps
th1 = mean_vsw[0,:,:]          # Mean Volumetric Soil Moisture for Layer 1
th2 = mean_vsw[1,:,:]          # Mean Volumetric Soil Moisture for Layer 2

We need to have the values for the first and second soil depths

Layer 2 includes Layer 1 so we need to remove 2 from 1

All Layers are the same through out so we use step 1

In this instance the shape is ds.sod['step', 'level', 'y', 'x']

In [ ]:
sd1 = ds.sod[0,0,:,:].values
sd2 = ds.sod[0,1,:,:].values - ds.sod[0,0,:,:].values

Now we calculate the mean wilting point

In [ ]:
data=(thmin1.wiltingpoint.values * sd1 + thmin2.wiltingpoint.values * sd2)/(sd1 + sd2)
thmin = xr.DataArray(data,dims=(ds.y.name,ds.x.name),name='thmin')
ds['thmin'] = thmin

Now we calculate the field capacity

In [ ]:
data=(thmax2.fieldcapacity.values * sd1 + thmax2.fieldcapacity.values * sd2)/(sd1 + sd2)
thmax = xr.DataArray(data,dims=(ds.y.name,ds.x.name),name='thmax')
ds['thmax'] = thmax

Now we calculate the mean soil moisture for layer 1 and 2

In [ ]:
data=((th1 * sd1) + (th2 * sd2))/(sd1 +sd2)
smtot = xr.DataArray(data,dims=(ds.y.name,ds.x.name),name='smtot')
ds['smtot'] = smtot 

Now we calculate the soil wetness index

In [ ]:
data=(((smtot - thmin)/(thmax - thmin)))
SM = xr.DataArray(data,dims=(ds.y.name,ds.x.name),name='SM')
ds['SM'] = SM

In [ ]:
def make_cmap(colors, position=None):
    '''
    make_cmap takes a list of tuples which contain RGB values. 
    The RGB values are [0 to 255] 
    make_cmap returns a cmap with equally spaced colors.
    Arrange your tuples so that the first color is the lowest value for the
    colorbar and the last is the highest.
    position contains values from 0 to 1 to dictate the location of each color.
    '''
    import matplotlib as mpl
    import numpy as np
    import sys
    bit_rgb = np.linspace(0,1,256)
    if position == None:
        position = np.linspace(0,1,len(colors))
    else:
        if len(position) != len(colors):
            sys.exit("position length must be the same as colors")
        elif position[0] != 0 or position[-1] != 1:
            sys.exit("position must start with 0 and end with 1")
    for i in range(len(colors)):
        colors[i] = (bit_rgb[colors[i][0]],
                     bit_rgb[colors[i][1]],
                     bit_rgb[colors[i][2]])
    cdict = {'red':[], 'green':[], 'blue':[]}
    for pos, color in zip(position, colors):
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))

    cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)
    return cmap


In [ ]:
%matplotlib notebook
from matplotlib.pyplot import colorbar
import matplotlib.pyplot as plot
from mpl_toolkits.basemap import Basemap,cm
import numpy as np
import pandas as pd
plot.figure(figsize=(10,10),num='EFAS Data in matplotlib with Jupyter')

m = Basemap(width=5400000,height=5000000,
            resolution='l',projection='laea',\
            lat_ts=72,lat_0=50,lon_0=18.)

m.drawparallels(np.arange(-80.,81.,20.),labels=[False, True, True, False])
m.drawmeridians(np.arange(-180.,181.,20.), labels=[True,False,False,True])
m.drawmapboundary(fill_color='white')
m.drawcoastlines(color='grey')
m.drawcountries(color='grey')
m.fillcontinents()

colors = [(64,0,3), (133,8,3), (249,0,0), (250,106,0), (249,210,0), (189,248,255), (92,138,255), (0,98,255), (0,0,255), (0,0,88)]
### Create an array or list of positions from 0 to 1.
position = [0, 0.2, 0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

cmap = make_cmap(colors, position=position)
#cmap.set_under('white')
x,y=m(ds.longitude.values,ds.latitude.values)
sc = plot.contour(x,y,ds.SM[:,:].values,300,vmin=0,cmap=cmap,vmax=1)
plot.title('Mean Soil Moisture over 10 days\n BASETIME : ' +  np.datetime_as_string(ds.time.values,'h'))
cbar = colorbar(shrink=0.5,ticks=np.linspace(0.1,1.0,10,endpoint=True))
cbar.set_label(ds.SM.name)
plot.show()